### Import scanpy

In [1]:
import scanpy as sc
import scanpy.external as sce
#sc.logging.print_versions()
#sc.logging.print_memory_usage()
#sc.settings.verbosity = 2
import os,sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib.ticker as mticker

In [2]:
# add the utility function folder to PATH
sys.path.append(os.path.abspath("utility_functions/"))

from rz_import_statements import *
import rz_functions as rz
import rz_fig_params # this adjust mpl.rcParams, almost nothing to import, import after scanpy to overwrite rc.Params
import rz_utility_spring as srz

python version: 3.11.7


# Load data

In [4]:
adata = sc.read_h5ad('backups/final_sc_hvg_palantir_15372x3324_231130_12h58.h5ad') 

In [5]:
# overwrite obs with the most recent version
filename = 'backups/obs_info_annotated_sc_hvg_no_cycle_15372x25_231129_17h30.npz'
encoding = 'latin1'

with np.load(filename,encoding=encoding, allow_pickle = True) as f:
    obs = pd.DataFrame(**f)
adata.obs = obs

### Scale (normalize) data

In [7]:
adata = adata.raw.to_adata()

In [8]:
# turn into counts per 10k
print(adata.X[:5,:].sum(axis=1))
print()
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
print(adata.X[:5,:].sum(axis=1))

[[ 8560.]
 [19847.]
 [10175.]
 [ 7250.]
 [15370.]]

[[10000.   ]
 [10000.001]
 [10000.   ]
 [ 9999.999]
 [10000.001]]


# Get enriched genes

Upon interactive exploration of the UMAP in the SPRING (Weinreb et al., 2018) application, it was decided that spectral clustering with parameter k=43 is the best representation to separate the cellular phenotypes present in the dataset.

In [9]:
#taking all cells now

cmask = np.repeat(True,adata.shape[0])
print(cmask.sum())

15372


In [11]:
#spectral clustering labels
for i in adata.obs:
    if i.startswith('cell_type'):
        print (i)

cell_type


In [12]:
# get centroids selecting the cluster configuration with k=43
thelabel = 'cell_type'
centroids = rz.centroids(thelabel,adata[cmask])

In [13]:
# For each cluster, find genes that are statistically significantly higher or lower in cluster x compared to
# all other cells collectively

#label-free filter to remove low abundance genes - 
#gene has to be expressed in at least min_cells by at least min_counts
min_counts = 10
min_cells = 5

In [14]:
gmask = srz.filter_abund_genes(adata.X[cmask], min_counts, min_cells)

3091 genes passing abundance filter


In [15]:
mwu_dict = {}
start=time.time()
counter=0

meta = adata[cmask].obs
E = adata[cmask].X
gene_list = adata.var_names


for cluster in meta[thelabel].unique():
    counter+=1
    mask1 = (meta[thelabel]==cluster).values
    mask2 = mask1==False
    
    cg1 = np.array(E[:,gmask][mask1,:].todense())
    cg2 = np.array(E[:,gmask][mask2,:].todense())
    mwu_dict[cluster] = rz.mwu(cg1,cg2,genes=gene_list[gmask],print_progression=True)
    print("%d/%d"%(counter,len(meta[thelabel].unique())))
    print(cluster, 'done',cg1.shape[0]+cg2.shape[0])
print(time.time()-start)

fname = 'backups/cell_types_vs_rest_MWU_result_dict_no_cycle_%s%s'%(rz.now(), thelabel)
print(fname)
rz.save_stuff(mwu_dict,fname)

1000
2000
3000
1/9
HSC done 15372
1000
2000
3000
2/9
CLP done 15372
1000
2000
3000
3/9
NMP done 15372
1000
2000
3000
4/9
Ery done 15372
1000
2000
3000
5/9
MPP done 15372
1000
2000
3000
6/9
pDC done 15372
1000
2000
3000
7/9
Mega done 15372
1000
2000
3000
8/9
cDC done 15372
1000
2000
3000
9/9
Mast done 15372
182.83367085456848
backups/cell_types_vs_rest_MWU_result_dict_no_cycle_231218_18h00cell_type


In [14]:
# if continuing from backup
#mwu_dict = rz.load_stuff('backups_JZ_2022/cluster_vs_rest_MWU_result_dict_220317_17h32sp_cl_43.pickle')

In [16]:
# select pseudovalue to add
pseudo = 1 # 1 counts per 10k

In [17]:
thelabel = 'cell_type'

In [18]:
print(thelabel)

# fcdict - fold-change dictionary

fcdict = rz.get_fc_to_all_other(
        lab = thelabel,
        meta = adata[cmask].obs,
        E = adata[cmask].X,
        pseudo = pseudo,
        gene_list = adata.var_names,
        )

cell_type


In [19]:
# leave only genes with a significant difference.
fcdictsig = {}

# before the mwu test, I prefiltered genes on abundance, apply this mask here as well
print(gmask.sum())
for key,value in fcdict.items():
    sigmask = (mwu_dict[key]['fdr']<0.05).values
    fcdictsig[key] = value[gmask][sigmask]
    print(key,sigmask.sum(),len(fcdictsig[key]))

3091
HSC 2922 2922
CLP 2668 2668
NMP 2356 2356
Ery 2708 2708
MPP 2560 2560
pDC 1858 1858
Mega 2176 2176
cDC 1889 1889
Mast 1071 1071


In [20]:
# nr genes to consider:
upto = 100 # up to 100 genes used to generate Table S1


frame = {}
for key,value in fcdictsig.items():
    s = value.sort_values(ascending=False)[:upto]
    key2 = str(key)+'_FC'
    frame[str(key)] = s.index
    frame[key2] = s.values
frame = pd.DataFrame(frame)
frame[[i for i in frame.columns if "FC" in i]].min() #ok, all above 1.

HSC_FC     1.303967
CLP_FC     1.959370
NMP_FC     1.469594
Ery_FC     1.355569
MPP_FC     1.226320
pDC_FC     1.864269
Mega_FC    1.483150
cDC_FC     1.784239
Mast_FC    1.420517
dtype: float32

In [21]:
outdir = 'outputs/'

In [22]:
fname = outdir+'lists_enriched_genes_no_cycle_top_%d_%s_%s.xlsx'%(upto,thelabel,rz.now())
print(fname)
frame.to_excel(fname)

outputs/lists_enriched_genes_no_cycle_top_100_cell_type_231218_18h03.xlsx
